In [2]:
#### import the simple module from the paraview
from paraview.simple import *
#### disable automatic camera reset on 'Show'
paraview.simple._DisableFirstRenderCameraReset()

ModuleNotFoundError: No module named 'paraview'

In [2]:


# Create a new 'Render View'
renderView1 = GetRenderView()
renderView1.Background = [0.3686274509803922]*3

fname = '/scratch/snx3000/farbodh/Lightcone_data/LCDM_lightcones/lcdm_lightcone1_0104_cdm'

reader = LightConeSeriesReader(registrationName='lcdm_lightcone1', FileNames=[fname])
reader.PointArrayStatus = ['velocity', 'id']
reader.DistributedSnapshot = 0
reader.UpdatePipeline()

programmableFilter1 = ProgrammableFilter(registrationName='ProgrammableFilter1', Input=reader)
programmableFilter1.OutputDataSetType = 'Same as Input'
programmableFilter1.Script = """
from vtk.vtkCommonDataModel import vtkPolyData
from vtk.vtkCommonCore import vtkIdList
from vtk.numpy_interface import dataset_adapter as dsa
from vtk import VTK_POLY_VERTEX
import numpy as np

Ratio = 1000

def SubSampler(input, output):
  my_input = dsa.WrapDataObject(input)
  nbpts = input.GetNumberOfPoints()
  indices = np.arange(0, nbpts, Ratio)
  nnodes = indices.size
  print("found ", nbpts, ". Extracting ", nnodes)
  ptIds = vtkIdList()
  ptIds.SetNumberOfIds(nnodes)

  my_output = dsa.WrapDataObject(output)
  for a in range(nnodes):
    ptIds.SetId(a , a)

  output.Allocate(1)
  output.InsertNextCell(VTK_POLY_VERTEX , ptIds)
  my_output.Points = my_input.Points[indices]
  my_output.PointData.append(my_input.PointData["velocity"][indices], "velocity")
  my_output.PointData.append(my_input.PointData["id"][indices], "id")
  
if inputs[0].IsA("vtkMultiBlockDataSet"):
    output.CopyStructure(inputs[0].VTKObject)
    iter = inputs[0].NewIterator()
    iter.UnRegister(None)
    iter.InitTraversal()
    while not iter.IsDoneWithTraversal():
        curInput = iter.GetCurrentDataObject()
        curOutput = curInput.NewInstance()
        curOutput.UnRegister(None)
        output.SetDataSet(iter, curOutput)
        SubSampler(curInput, curOutput)
        iter.GoToNextItem();
else:
  SubSampler(input, output)"""

rep1 = Show(programmableFilter1, renderView1, 'GeometryRepresentation')
rep1.Representation = 'Points'
ColorBy(rep1, ('POINTS', 'velocity', 'Magnitude'))

myselect2 = QuerySelect(QueryString="mag(velocity) >= 1000.", Source=reader)
sel2     = ExtractSelection(Input=reader)

rep2 = Show(sel2, renderView1, 'GeometryRepresentation')
rep2.Representation = 'Points'
ColorBy(rep2, ('POINTS', 'velocity', 'Magnitude'))

ResetCamera()
# use the current default object just defined above
SaveData("/scratch/snx3000/jfavre/clip3.csv")


Using a terminal on this node, we request a new allocation:

salloc -N 64 -n 64-A csstaff -C gpu --time=00:15:00

salloc: Granted job allocation 30807331 salloc: Waiting for resource configuration salloc: Nodes nid0[3508-3511] are ready for job

We *first* run pvserver on the newly allocated nodes

srun pvserver

The jupyter client must connect to the first MPI task on the new allocation, i.e. connect to the first node

In [3]:
Connect("nid01980")

Connection (cs://nid01980:11111) [2]

In [4]:
LoadPlugin("/users/jfavre/Projects/Adamek/ParaViewLightConePlugin/build59/lib64/paraview-5.9/plugins/pvLightConeReader/pvLightConeReader.so", ns=globals())

In [5]:
renderView1 = GetRenderView()

reader = LightConeSeriesReader(FileNames=['/scratch/snx3000/farbodh/Gadget2_snapshot/output_redist/lcdm_Mn0d13_gevolution_snap000_cdm_redist.0'])
reader.PointArrayStatus = ['velocity']
reader.UpdatePipelineInformation()

In [6]:
reader.DistributedSnapshot = 0 # to read a single file (you should be able to run on the single node already allocated)
reader.DistributedSnapshot = 1 # to read all files (you must do a second node allocation

In [7]:
reader.UpdatePipeline()

In [8]:
from ipyparaview.widgets import PVDisplay
disp = PVDisplay(GetActiveView())
w = display(disp)

PVDisplay(resolution=(400, 400))

In [9]:
clip1 = Clip(Input=reader)
clip1.ClipType = 'Sphere'
clip1.ClipType.Center = [600000.0, 600000.0, 600000.0]
clip1.ClipType.Radius = 200000.0

In [10]:
repr2 = Show(clip1)

repr2.Representation = 'Points'
ColorBy(repr2, ('POINTS', 'velocity', 'Magnitude'))
ResetCamera()

In [11]:
Disconnect()

In [1]:
# We can d the same thing using python script and srun 
# https://user.cscs.ch/computing/visualisation/paraview/#interactive-mode-with-a-client-server-connection